In [3]:
#EUREC4A_HALO_BAHAMAS-Radiometer_20200122a.nc
#EUREC4A_HALO_BAHAMS-QL-10Hz_20200122a.nc
#Nav_IMS0000.Asc
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import Hodograph, SkewT
from metpy.units import units
my_example_nc_file = 'EUREC4A_HALO_BAHAMAS-Radiometer_20200202a.nc'
fh = Dataset(my_example_nc_file, mode='r')
df_fdir = pd.read_csv('BBR_DirectFraction_Flight_20200202a_R0.dat',skiprows = 34,delim_whitespace=True)
df_fdw = pd.read_csv('BBR_Fdn_clear_sky_Flight_20200202a_R0.dat',skiprows = 34, delim_whitespace=True)
fdw_time = df_fdw.loc[:,['sod','F_dw']]
fdir_time = df_fdir.loc[:,'f_dir']
db = pd.concat([fdw_time,fdir_time],axis=1)
db.set_index('sod',inplace=True)
time = fh.variables['TIME'][:] # time
time_bac = pd.DataFrame(time.T, columns = ['time'])
#pd.options.display.float_format = '{:,.0f}'.format
time_bac = time_bac/1000
time_bac.head()
#print(fh.variables.keys())
#db

,time
0,41282.0
1,41282.1
2,41282.2
3,41282.3
4,41282.4


In [9]:
nav = pd.read_csv('Nav_IMS0000.Asc',skiprows=14, header=None,delim_whitespace=True)
#nav.drop(['B', 'C'], axis=1)
nav.iloc[:,2:5]

,2,3,4
0,-0.204582,1.390774,76.065269
1,-0.204607,1.390832,76.065315
2,-0.204657,1.390752,76.065445
3,-0.204877,1.390615,76.065475
4,-0.204955,1.390674,76.065605
...,...,...,...
751426,3.414081,-2.702178,-78.576035
751427,3.411791,-2.700373,-78.576698
751428,3.413457,-2.702226,-78.577835
751429,3.411241,-2.699315,-78.578369


In [2]:
def convertSeconds(seconds):
    h = seconds//(60*60)
    m = (seconds-h*60*60)//60
    s = seconds-(h*60*60)-(m*60)
    return [h, m, s]
convertSeconds(41282)

[11, 28, 2]

In [ ]:
array = np.arange(41282,72804, 1)
arr = array.repeat(10)
df_arr = pd.DataFrame(arr, columns = ['time_arr'])
df_arr.set_index('time_arr',inplace=True)
data = df_arr.join(db)
data.reset_index(level=0, inplace=True)
dat = pd.concat([time_bac,data],axis=1)
dat = dat.drop(columns=['index'])
dat_dup = dat.drop_duplicates('F_dw')
dat_dup.columns = ['t','f_dw','f_dr']
df_big = pd.concat([dat,dat_dup],axis=1)
df_bacardi = df_big.drop(['t','F_dw','f_dir'],axis=1)
df_bacardi.columns = ['Time','F_dw','f_dir']
df_final = df_bacardi.interpolate(method='nearest', axis=0).ffill().bfill()
df_final
#df_final.to_csv('df_final.txt')

In [ ]:
ri = fh.variables['IRS_PHI'][:] # roll angle "deg"
ro = pd.DataFrame(ri.T)
pi = fh.variables['IRS_THE'][:] # pitch angle "deg"
po = pd.DataFrame(pi.T)
yi = fh.variables['IRS_HDG'][:] # yaw angle "deg"
y = pd.DataFrame(yi.T)
szai = fh.variables['SUN_ALT'][:] # solar altitude angle
sza = pd.DataFrame(szai.T)
saai = fh.variables['SUN_AZI'][:] # solar azimuth angle
saa = pd.DataFrame(saai.T)
fdw = df_final.loc[:,'F_dw']
fdir = df_final.loc[:,'f_dir']
SWd = fh.variables['FDSC'][:]
SWD = pd.DataFrame(SWd.T)
SWD.columns=['swu']
r_off = 0.3
p_off = -2.55
r = ro+r_off
p = po+p_off

In [ ]:
import math
def Fdw_attitude_correction(fdw,r,p,y,sza,saa,r_off,p_off,fdir):
    for i in sza.iloc[:,0:1]:
        for j in saa.iloc[:,0:1]:
            for k in r.iloc[:,0:1]:
                for l in p.iloc[:,0:1]:
                    for m in y.iloc[:,0:1]:
                        for n in fdw:
                            for o in fdir:
                                eq1 = math.sin((90-i)*0.0175)
                                eq2 = (math.cos((90-i)*0.0175)*math.sin(k*0.0175)*math.sin((j-m)*0.0175))
                                eq3 = (math.cos((90-i)*0.0175)*math.sin(l*0.0175))
                                eq4 = (math.cos(k*0.0175)*math.cos((j-m)*0.0175))
                                eq5 = (math.sin((90-i)*0.0175)*math.cos(l*0.0175)*math.cos(k*0.0175))
                                #factor = eq1/((eq2-eq3*(eq4+eq5))
                                factor = eq1/(eq2-((eq3+eq4)+eq5))
                                Fdw=o*n*factor+(1-o)*n

correction_weighted = Fdw_attitude_correction(fdw,r,p,y,sza,saa,r_off,p_off,fdir)

corrected_FDSC = SWD.swu - correction_weighted
swd_correct = pd.DataFrame(corrected_FDSC)
swd_correct.columns = ['swd_corr']
swd_correct

In [ ]:
df_final.Time

In [ ]:
import datetime 
  
def convert(n): 
    return str(datetime.timedelta(seconds = n)) 
Dtime = []
for x in df_final.Time:
    Dtime.append(convert(x))


In [1]:
# def smooth(y, box_pts):
#     box = np.ones(box_pts)/box_pts
#     y_smooth = np.convolve(y, box, mode='same')
#     return y_smooth
fg = pd.concat([df_final.Time,swd_correct.swd_corr,SWD.swu],axis=1)
plt.figure(figsize=(16,8),dpi=500)
plt.title('BACARDI F_dw from the flight on 02-02-2020', fontsize =14)
plt.plot(fg.Time,fg.swu, 'b-', label = 'solar radiation up (raw)')
plt.plot(fg.Time,fg.swd_corr, 'r-', label = 'solar radiation up (attitude_corrected)')
#plt.plot(fg.Time, smooth(fg.swd_corr,500), 'b-')
#plt.plot(df_time_delta.time,bacardi.ShortWave_DOWN, 'm', label = 'solar radiation down')
#plt.plot(df_time_delta.time,bacardi.LongWave_UP, 'y', label = 'terrestrial radiation up')
#plt.plot(df_time_delta.time,bacardi.LongWave_DOWN, 'c', label = 'terrestrial radiation down')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)
plt.xlabel('Time (UTC)',fontsize=14)
plt.ylabel('Irradiance ($W/M^2$)',fontsize=14)
#plt.ylim(40,1280)
plt.show()
#corrected_FDSC.to_csv('corrected_FDSC.csv')


NameError: name 'pd' is not defined

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth
plt.figure(figsize=(15,6),dpi=500)
#plt.plot(fg.Time,fg.swd_corr,'o')
plt.plot(fg.Time, smooth(fg.swd_corr,3), 'r-', lw=2)
plt.plot(fg.Time, smooth(fg.swd_corr,19), 'g-', lw=2)
plt.show()

In [ ]:

#pd.to_datetime('2016-06-10 18:57:35:317', format="%Y-%m-%d %H:%M:%S:%f")
# import datetime
# start = datetime.datetime(2020, 1, 30, 11, 19, 34)
# end = datetime.datetime(2020, 1, 30, 15, 8, 21)
# delta = end - start
# time_delta = []
# for i in range(delta.seconds + 1):
#     time_delta.append(start + datetime.timedelta(seconds=i))
# df_time_delta = pd.DataFrame(time_delta)
# df_time_delta.columns = ['time']
# df_time_delta.shape

In [ ]:
def convertSeconds(seconds):
    h = seconds//(60*60)
    m = (seconds-h*60*60)//60
    s = seconds-(h*60*60)-(m*60)
    return [h, m, s]
convertSeconds(87032)

In [ ]:
T/1000

In [ ]:
FDSC

In [ ]:
# SWu = fh.variables['FDSC'][:]
# SWd = fh.variables['FUS'][:]
# LWu = fh.variables['FDL'][:]
# LWd = fh.variables['FUL'][:]
# time = fh.variables['TIME'][:]
# SWU = pd.DataFrame(SWu.T)
# SWD = pd.DataFrame(SWd.T)
# LWU = pd.DataFrame(LWu.T)
# LWD = pd.DataFrame(LWd.T)
# TIME = pd.DataFrame(time.T)/1000
# df = pd.concat([TIME,SWU,SWD,LWU,LWD],axis =1)
# df.columns = ['Time', 'ShortWave_UP', 'ShortWave_DOWN','LongWave_UP', 'LongWave_DOWN']
# df.to_csv('time_22jan.csv')

In [ ]:
# #array = np.linspace(53856,87031, num=33175)
# array = np.arange(53856,87031, 1)
# df_arr = pd.DataFrame(array, columns = ['time_arr'])
# fdw_time = df_fdw.loc[:,['sod','F_dw']]
# #fdw_time
# for g in fdw_time.sod:
#     for h in df_arr.time_arr:
#         if  g == h:
#             print (yes)
#         else:
#             print(no)

In [ ]:
# df_arr['time_arr_bool'] = df_arr.isin(fdw_time)
# # df_arr.set_index('Locality', inplace=True)
# # df_arr


In [ ]:
math.sin(21.3*0.0175)

In [ ]:
math.pi/180

In [ ]:
(22/7)/180

In [ ]:
*(math.pi/180)